In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
2,"Michigan adds 21 deaths, 2,909 more cases of C...",https://www.detroitnews.com/story/news/local/m...,"Mon, 19 Oct 2020 18:59:43 GMT",2020-10-19 18:59:43+00:00
1,Audio & Rush Transcript: Governor Cuomo Update...,https://www.governor.ny.gov/news/audio-rush-tr...,"Mon, 19 Oct 2020 17:28:48 GMT",2020-10-19 17:28:48+00:00
6,Delaware Announces Grant Program for Contract ...,https://news.delaware.gov/2020/10/19/delaware-...,"Mon, 19 Oct 2020 17:21:40 GMT",2020-10-19 17:21:40+00:00
9,Trump's campaigning stoops to new lows as Covi...,https://www.cnn.com/2020/10/19/politics/donald...,"Mon, 19 Oct 2020 16:10:00 GMT",2020-10-19 16:10:00+00:00
8,COVID-19 vaccine: UNICEF to stockpile more tha...,https://news.un.org/en/story/2020/10/1075682,"Mon, 19 Oct 2020 14:04:06 GMT",2020-10-19 14:04:06+00:00
0,COVID-19 Daily Update 10-19-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Mon, 19 Oct 2020 14:03:01 GMT",2020-10-19 14:03:01+00:00
3,Trends in Overall and Non-COVID-19 Hospital Ad...,https://www.kff.org/health-costs/issue-brief/t...,"Mon, 19 Oct 2020 12:56:33 GMT",2020-10-19 12:56:33+00:00
4,2021 Premium Changes on ACA Exchanges and the ...,https://www.kff.org/private-insurance/issue-br...,"Mon, 19 Oct 2020 11:07:45 GMT",2020-10-19 11:07:45+00:00
5,Latest on COVID-19 in MN: 12 days of cases abo...,https://www.mprnews.org/story/2020/10/19/lates...,"Mon, 19 Oct 2020 10:38:00 GMT",2020-10-19 10:38:00+00:00
7,COVID-19 Daily Update 10-18-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sun, 18 Oct 2020 14:01:52 GMT",2020-10-18 14:01:52+00:00
